In [1]:
from L420A import everything

In [2]:
everything.setup()

Biosphere database already present!!! No setup is needed
Database has already been imported


In [3]:
everything.constructor_main(file = 'Case_study_existing_building_wEOL_4.xlsx', sheet = 'Construction')

In [4]:
everything.renovation_main(file = 'Case_study_existing_building_wEOL_4.xlsx', sheet = 'Renovation')

In [5]:
mfa = everything.Mfa(2020, 2050)

In [6]:
for product in everything.Product.real_instances:
    mfa.add_renovation_points(product)

In [7]:
for product in everything.Product.real_instances:
    mfa.add_eol_points(product)

In [8]:
mfa.points

[(2046, heat pump production),
 (2046, evacuated tube collector production),
 (2046, polyethylene production, high density, granulate),
 (2046, hot water tank production, 600l),
 (2046, steel production, converter, low-alloyed),
 (2046, ventilation duct production, steel, 100x50 mm),
 (2050, ventilation duct production, steel, 100x50 mm),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, fibre cement roof slate production),
 (2050, wood wool production),
 (2050, gypsum plasterboard production),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, market for plywood),
 (2050, gypsum plasterboard production),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, wood wool production),
 (2050, door production, outer, wood-aluminium),
 (2050, 

In [9]:
clean_list_renovation= mfa.clean_list_renovation()

In [10]:
clean_list_eol = mfa.clean_list_eol()

In [11]:
# extend the renovation list with the eol list
clean_list = clean_list_renovation + clean_list_eol

In [12]:
lca = everything.ProLCA(activities_and_years=clean_list, yearly_databases=mfa.create_time_line_dict(), 
                         methods=[('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)'), ('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)')],
                         products=everything.Product.real_instances)

In [17]:
operational = lca.give_me_operational()

26
4
45416.35306226133
45416.35306226133
5
56725.119398328825
56725.119398328825
3
33515.85636394306
33515.85636394306
22
241974.13394453467
241974.13394453467


In [14]:
# production_lci = lca.production_lca()

In [15]:
# other_lci = lca.give_me_embodied()

In [16]:
# # pickle other lci
# import pickle
# with open('other_lci.pickle', 'wb') as handle:
#     pickle.dump(other_lci, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # # pickle production lci
# # import pickle
# with open('production_lci.pickle', 'wb') as handle:
#     pickle.dump(production_lci, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
# unpickle production lci and other lci
import pickle
with open('production_lci.pickle', 'rb') as handle:
    production_lci = pickle.load(handle)
    
with open('other_lci.pickle', 'rb') as handle:
    other_lci = pickle.load(handle)

In [39]:
production_lci

{(2020,
  concrete_foundation_production,
  'ecoinvent-3.9.1-cuttoff'): [13872.880638500286, 43.99707920471239],
 (2020,
  polystyrene foam slab production, 45% recycled,
  'ecoinvent-3.9.1-cuttoff'): [3419.038665225468, 12.786087097917745],
 (2020,
  polyethylene production, low density, granulate,
  'ecoinvent-3.9.1-cuttoff'): [319.209695386447, 1.1410022020801207],
 (2020,
  fibreboard production, hard,
  'ecoinvent-3.9.1-cuttoff'): [202.85713958731665, 4.3870538907674375],
 (2020,
  steel production, low-alloyed, hot rolled,
  'ecoinvent-3.9.1-cuttoff'): [1819.3230344217818, 7.737518979739819],
 (2020,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates,
  'ecoinvent-3.9.1-cuttoff'): [0.0, 0.0],
 (2020,
  glass wool mat production,
  'ecoinvent-3.9.1-cuttoff'): [2473.579532791892, 26.075566322188127],
 (2020,
  gypsum plasterboard production,
  'ecoinvent-3.9.1-cuttoff'): [494.72569017911144, 3.0273015047543606],
 (2

In [17]:
other_lci

{(2046,
  'adc6bbccf3d5b2955b050510e647852a',
  'SSP2-RCP19_2040'): [5.167113395618682, 0.22785525580534982],
 (2046,
  'adc6bbccf3d5b2955b050510e647852a',
  'SSP2-RCP26_2040'): [5.708426064068332, 0.2285534684257243],
 (2046,
  '2c559dbc4e822b444792e122dffe6c20',
  'SSP2-RCP19_2040'): [-0.029349983716362896, -7.371097654687322e-05],
 (2046,
  '2c559dbc4e822b444792e122dffe6c20',
  'SSP2-RCP26_2040'): [-0.02978563976492274, -7.457237612359403e-05],
 (2046,
  '6032fcb0a208bdcdab5633d6be417d0e',
  'SSP2-RCP19_2040'): [60.96440817983701, 2.003524394973958],
 (2046,
  '6032fcb0a208bdcdab5633d6be417d0e',
  'SSP2-RCP26_2040'): [76.9201013399435, 2.042545346996283],
 (2046,
  '1580abc0d8542f9117775cc18e264f97',
  'SSP2-RCP19_2040'): [1.8316756421438505, 0.0059965357531730715],
 (2046,
  '1580abc0d8542f9117775cc18e264f97',
  'SSP2-RCP26_2040'): [1.862369610272013, 0.005953300993789006],
 (2046,
  '1c2dd8677907f88a0746b958ca4efd43',
  'SSP2-RCP19_2040'): [598.9502882688827, 2.881731206641944],
 

In [18]:
clean_list

[(2046,
  'adc6bbccf3d5b2955b050510e647852a',
  ventilation duct production, steel, 100x50 mm),
 (2046,
  '2c559dbc4e822b444792e122dffe6c20',
  ventilation duct production, steel, 100x50 mm),
 (2046,
  '6032fcb0a208bdcdab5633d6be417d0e',
  evacuated tube collector production),
 (2046,
  '2c559dbc4e822b444792e122dffe6c20',
  evacuated tube collector production),
 (2046,
  '1580abc0d8542f9117775cc18e264f97',
  polyethylene production, high density, granulate),
 (2046,
  '2c559dbc4e822b444792e122dffe6c20',
  polyethylene production, high density, granulate),
 (2046, '1c2dd8677907f88a0746b958ca4efd43', hot water tank production, 600l),
 (2046, '2c559dbc4e822b444792e122dffe6c20', hot water tank production, 600l),
 (2046,
  '89cb4e1a47b707fe43b99135b81fcaba',
  steel production, converter, low-alloyed),
 (2046,
  '2c559dbc4e822b444792e122dffe6c20',
  steel production, converter, low-alloyed),
 (2050, '6032fcb0a208bdcdab5633d6be417d0e', fibreboard production, hard),
 (2050, '557ae7fde4ce81515

In [19]:
def aggregation_renovation_eol():
    lci_renovation_eol = other_lci.copy()
    clean_list_copy = clean_list.copy()
    aggregated_results = {}
    # loop over the lci renovation eol dictionary and clean list
    # check if the activity matches the clean list is in the lci dictionary
    # multiply the lci dictionary with the clean list and add the results to a list
    for key, value in lci_renovation_eol.items():
        for i in clean_list_copy:
            
            if key[1] == i[1]:
                if len(value) > 1:
                    # multiply each value in the list by the amount
                    result = [x*i[2].amount for x in value]
                    aggregated_results[(i[0], i[2], key[2])] = result
    return aggregated_results
aggreat = aggregation_renovation_eol()

In [20]:
# append aggregated results to the production lci to a new dictionary
full_results_no_elec = {}
for key, value in production_lci.items():
    full_results_no_elec[key] = value
for key, value in aggreat.items():
    full_results_no_elec[key] = value
for key, value in operational.items():
    full_results_no_elec[key] = value
full_results_no_elec


{(2020,
  concrete_foundation_production,
  'ecoinvent-3.9.1-cuttoff'): [13872.880638500286, 43.99707920471239],
 (2020,
  polystyrene foam slab production, 45% recycled,
  'ecoinvent-3.9.1-cuttoff'): [3419.038665225468, 12.786087097917745],
 (2020,
  polyethylene production, low density, granulate,
  'ecoinvent-3.9.1-cuttoff'): [319.209695386447, 1.1410022020801207],
 (2020,
  fibreboard production, hard,
  'ecoinvent-3.9.1-cuttoff'): [202.85713958731665, 4.3870538907674375],
 (2020,
  steel production, low-alloyed, hot rolled,
  'ecoinvent-3.9.1-cuttoff'): [1819.3230344217818, 7.737518979739819],
 (2020,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates,
  'ecoinvent-3.9.1-cuttoff'): [0.0, 0.0],
 (2020,
  glass wool mat production,
  'ecoinvent-3.9.1-cuttoff'): [2473.579532791892, 26.075566322188127],
 (2020,
  gypsum plasterboard production,
  'ecoinvent-3.9.1-cuttoff'): [494.72569017911144, 3.0273015047543606],
 (2

In [ ]:
everything.GiveMeName.poop_to_excel(data, 'test.xlsx')

{'concrete_foundation_production': {(2050, '3d3a786bfd99f7e4e312c33c1def0b86', 'SSP2-RCP19_2050'): [-0.18662983177191184, -0.0011398314906943619], (2050, '3d3a786bfd99f7e4e312c33c1def0b86', 'SSP2-RCP26_2050'): [-0.18781225780212138, -0.0011384109012428127], (2050, '2eafb2856827730d501b1941671e3160', 'SSP2-RCP19_2050'): [12377.438986924704, 36.810137543811805], (2050, '2eafb2856827730d501b1941671e3160', 'SSP2-RCP26_2050'): [12402.154685428746, 36.8389514576937]}, 'polystyrene foam slab production, 45% recycled': {(2050, '3f058e3bf6791c71e3b121c63c75eba4', 'SSP2-RCP19_2050'): [-364.3300248587018, -0.43636715518737706], (2050, '3f058e3bf6791c71e3b121c63c75eba4', 'SSP2-RCP26_2050'): [-364.35692435565386, -0.4363232939180831], (2050, '2d1a6f71ee1b61ae0650e97319e5fad2', 'SSP2-RCP19_2050'): [3268.8435189827937, 12.041544468207517], (2050, '2d1a6f71ee1b61ae0650e97319e5fad2', 'SSP2-RCP26_2050'): [3305.2727003051937, 11.991370094950478]}, 'polyethylene production, low density, granulate': {(2050